En este notebook empezaremos con un micro-curso de probabilidad usando nuestra nueva herramienta. En este momento es importante recalcar que a medida que vayamos avanzando en los notebooks, las contribuciones en código de los autores serán cada vez menores, puesto que consideramos que lo aprendido en la primera parte del curso, más la sencillez de PyCUDA proveen al lector las herramientas necesarias para poder hacer sus propios códigos desde 0.

## Águila o sol (cara o cruz)

Al lanzar una moneda, uno puede decir con absoluta certeza que sólo una de las dos caras de la moneda verá hacia el frente, e incluso ir más allá: la probabilidad de caiga en cualquiera de las dos caras es de $\frac{1}{2}$.

Recomendamos leer sobre la visión "frecuentista" y "bayesiana" de las probabilidades que se le puede adjudicar a un evento. (Revisar las referencias)


**Definición**: Llamamos Distribución de Probabilidad a la colección de todas las probabilidades de todos los eventos de una variable aleatorio $X$ asociada a un evento.

Ejemplo: Al lanzar una moneda obtenemos dos posibles eventos. 
+ $X=$ Águila
+ $X=$ Sol

De este manera obtenemos que la Distribución de Probabilidad asociada al lanzar la moneda es: $$\{ P(X = Águila) = \frac{1}{2}, \ P(X=Sol)= \frac{1}{2} \}$$

En este ejemplo, las probabilidades son iguales en cada valor de $X$, por lo que llamamos a esta distribución *uniforme*. Pero, ¿qué tan uniforme es?

##  El dado

Cambiaremos ahora a otro ejemplo un poco más complicado: el lanzamiento de un dado de 6 caras. No es difícil llegar a que, si el dado no está trucado, cada una de las caras tiene una probabilidad de $\frac{1}{6}$ de quedar mirando hacia arriba.

Pasemos ahora a la programación.

Como primer objetivo haremos que $N$ `threads` generen un número entero aleatorio entre $1$ y $6$.

In [25]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.curandom import rand as curand
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray

# Escribimos el kernel que utilizaremos

# El kernel toma un arreglo con N números aleatorios entre 0 y 1 y los convierte en números aleatorios
# entre 1 y 6. 

mod = SourceModule("""
#include<stdio.h>
#include<math.h>

__global__ void Dado( float * d_arregloIn, float * d_arregloOut) {

    int idx = blockDim.x * blockIdx.x + threadIdx.x ;

    d_arregloOut[idx] = ceil( 6*d_arregloIn[idx] ) ;

}
""")

# definimos el número de dados a lanzar

N = 50

# Iniciamos los arreglos en el GPU

# CuRAND es una librería especial para lanzar números aleatorios y hacer cálculos estadísticos

a_gpu = curand(N,) # Ver la nota al final del Notebook
b_gpu = gpuarray.zeros_like(a_gpu)

# Rescatamos la función para luego ejecutarla

Dado = mod.get_function("Dado")

# El kernel es lanzado de forma muy parecida a CUDA C

Dado(a_gpu, b_gpu, block = (N, 1, 1), grid = (1, 1, 1))

print b_gpu

# Finalmente liberamos la memoria

a_gpu.free()
b_gpu.free()

[ 3.  6.  3.  4.  5.  6.  5.  4.  5.  3.  2.  2.  6.  1.  2.  4.  5.  4.
  6.  5.  2.  3.  2.  3.  2.  4.  6.  3.  1.  3.  5.  4.  5.  3.  4.  3.
  1.  6.  6.  1.  2.  6.  5.  6.  4.  4.  1.  5.  2.  6.]


En este primer código creamos únicamente un lanzador de $N$ dados de $6$ caras. Ahora pasemos a calculos un poco más interesantes.

**[1]** Como primer ejercicio tendrás que modificar el código de arriba de tal forma que *en el GPU* se calculen las frecuencias de cada uno de los valores obtenidos. El resultado final tiene que ser un `arreglo` en el cual estén las frecuencias.

Puedes basarte en el siguiente código de Python

```Python

F = np.zeros(6)
N = 50

for i in resultado
    for j in 1:6
        if i == j
            F[i] += 1./N
            
```

Puedes graficar estos valores en un histograma usando la funcion 
```Python 
np.hist()
``` 
para luego repetir el experimento con el mismo valor de $N$ y luego aumentando el valor.

¿Qué observas?

**[2]** En el histograma del ejercicio pasado se logra observar cómo entre mayor es el número de tiros, más se acerca la frecuencia de cada cara a la frecuencia teórica ($\frac{1}{6}$). Ahora tu trabajo será de calcular la desviación entre la frecuencia obtenida y el valor teórico. Evidentemente este cálculo debe de hacerse en el `GPU`. 

Ahora buscamos observar como graficar observar la desviación entre la frecuencia obtenida y su valor teórico. Al graficar directamente la desviación obtenemos una gráfica en la que no se observa mucho puesto que hay un decaimiento muy rápido para luego converger hacia 0. Sin embargo, al graficar el logaritmo de la desviación obtenemos este tipo de comportamiento el cual nos revela el caracter exponencial de la desviación con respecto al número de tiros.

<img src=Imagenes/Desviacion.png align=center>

###  Ejercicio teórico

(a) Pensemos sólo en el lado $1$. Sea $p := \frac{1}{6}$ la probabilidad de que caiga en este lado.
Si lanzamos 2 veces, ¿cuál es la probabilidad de que salgan (i) cero veces $1$; (ii) una vez $1$; (iii) dos veces $1$?

(b) Si lanzamos 3 veces, ¿cuáles son las probabilidades de que salga cada número posible de $1$s entre 0 y 3?

(c) Si lanzamos N veces, ¿cuáles son las probabilidades?

(d) ¿Cuál es el valor *promedio* que sale? ¿Qué otra cantidad podríamos querer calcular?

## Distribución binomial

Ahora pasemos a otro tipo de distribución de probabilidades basada en el **ensayo de Bernoulli**. La idea principal es sencilla: Un cierto evento tiene una probabilidad $p$ de éxito. Si el evento es exitoso, entonces se marca con el valor $1$. Sino, se obtiene el valor $0$.

Tomemos el ejemplo del dado. El evento exitoso será el obtener la cara con valor $1$, lo cual quiere decir que tenemos una probabilidad $p = \frac{1}{6}$ de que nuestro evento sea exitoso.

Al repetir el lanzamiento $N$ veces, entonces podemos obtener la *distribución de probabilidad* $Y_{N}$, la cual nos dará la colección de todas las probabilidades de que el evento sea exitoso cuando se repite $i$ veces el experimento. $i = 1, ..., N$

Para todo lo siguiente supondremos que nuestro evento exitoso será obtener un $6$ a la hora de tirar un dado.

**[3]** Ahora pasemos a algo un poco más complicado. Escribe un `kernel` que haga $N$ ensayos de Bernoulli y que calcule la proporción de eventos exitosos.

Para este problema tendremos distintas manera de obetener el resultado **dentro** del `GPU`(en el `CPU` el problema es trivial). 

La manera fácil y no eficiente es escribir un `kernel` en el cual sumemos un $1$ si el evento fue exitoso. Esto puede hacerse con un condicional `if`. 

En el caso en el cual nos decantemos por algo más eficiente, tendremos que recurrir a un `kernel` más sofisticado. El tipo de operación que se hará se llama "reducción" y ha sido ampliamente discutido a lo largo de blogs y foros. 

La idea es esta. Se dividirá el arreglo en el cual se tiene la información del éxito de los eventos en bloques de una cierta dimensión. En cada uno de estos bloques se sumarán las entradas (ceros y unos) y el resultado estará en la primera entrada de cada bloque. 

En un segundo tiempo, el primer bloque será llenado con los resultados de los demás bloques, obteniendo de nuevo un número de bloques llenos de información. La suma se repetirá de nuevo, y así el procedimiento hasta obtener un solo resultado con la suma de todos los bloques. 

Como ejemplo, supongamos un arreglo con 18 entradas que separaremos en 5 bloques de 4 entradas. Después del primer paso obtendremos que la primera entrada de cada uno de los 5 bloques será la suma de sus 4 correspondientes entradas. 

Con estos 5 resultados podemos llenar entonces un bloque de 4 entradas y tener otro con una sola entrada distinta de 0. Hacemos de nuevo el mismo procedimiento, para obtener en el primer bloque la suma de los cuatro resultados. La entrada del segundo bloque quedó intacta.

Finalmente, sólo habrá que sumar las últimas dos entradas restantes para obtener el resultado deseado.

Si el lector desea implementar y aprender esta herramienta, en las referencias colocamos un documento de NVIDIA en el cual se explica paso a paso un kernel de reducción muy sofisticado. Esta alternativa es altamente recomendable por todos los usos que se le puede dar.

**[4]** Finalmente calcula la distribución de probabilidad $Y_N$ con los resultados obtenidos en el inciso anterior.

Aquí te recomendamos usar distintos `kernels`, uno `__device__` con el que calcules la proporción de eventos exitosos, y otro `__global__` con el que calcules $Y_N$.

La distribución de probabilidad de $Y_{N,p}$ se llama una *distribución binomial*. Se dice que es una distribución de probabilidad *discreta*, ya que los valores que toma son valores discretos. 

La notación que se utiliza es

$$X \sim B(N,p)$$

para decir que $X$ es una variable aleatoria con distribución de probabilidad binomial con parámetros $N$ y $p$.

## Nota sobre cuRAND en PyCUDA

En este primer Notebook, y a partir de ahora usaremos la librería cuRAND la cual nos permite generar números aleatorios para cada `thread` *dentro* del `GPU`. Esto tiene grandes ventajas con respecto a la utilización de la función `rand()` dentro de `numpy`, sobre todo debido al mayor control que nos da cuRAND sobre los generadores. Sin embargo, cuRAND tiene la desventaja que el máximo número de generadores activos en `GPU`'s tipo Tesla es de 256. Para el caso de `GPU`'s tipo Fermi, 1024 generadores pueden ser activados sin -en principio- tener mayor problema.

**Recomendamos** ampliamente leer las dos primeras partes del Apéndice 3 para aprender un poco más sobre como se generan números pseudo-aleatorios y cuasi-aleatorios en el GPU con ayuda de cuRAND.

## Referencias

+ Wiki de [Probabilidad Bayesiana](https://en.wikipedia.org/wiki/Bayesian_probability)
+ Wiki de [Probabilidad Frecuentisca](https://en.wikipedia.org/wiki/Frequentist_probability)
+ WIki de [Ensayo de Bernoulli](https://es.wikipedia.org/wiki/Ensayo_de_Bernoulli) y de la [Distribución Binomial](https://en.wikipedia.org/wiki/Binomial_distribution)
+ Documento de NVIDIA donde se expone un [kernel de reducción](http://developer.download.nvidia.com/compute/cuda/1.1-Beta/x86_website/projects/reduction/doc/reduction.pdf)